In [0]:
iris = spark.read.csv("/FileStore/tables/iris.csv",inferSchema=True, header=True)
print(iris.count())
iris.show(5)

150
+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
+-----------+----------+-----------+----------+-----------+
only showing top 5 rows



In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
asb = VectorAssembler(inputCols=["sepallength", "sepalwidth", "petallength", "petalwidth"], outputCol="independente")
irisas = asb.transform(iris)
irisas.show(5)

+-----------+----------+-----------+----------+-----------+-----------------+
|sepallength|sepalwidth|petallength|petalwidth|      class|     independente|
+-----------+----------+-----------+----------+-----------+-----------------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|
+-----------+----------+-----------+----------+-----------+-----------------+
only showing top 5 rows



In [0]:
from pyspark.ml.feature import StringIndexer

In [0]:
ind = StringIndexer(inputCol="class", outputCol="dependente")
irisas = ind.fit(irisas).transform(irisas)
irisas.show()

+-----------+----------+-----------+----------+-----------+-----------------+----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|     independente|dependente|
+-----------+----------+-----------+----------+-----------+-----------------+----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|       0.0|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|       0.0|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|       0.0|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|       0.0|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|       0.0|
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|[5.4,3.9,1.7,0.4]|       0.0|
|        4.6|       3.4|        1.4|       0.3|Iris-setosa|[4.6,3.4,1.4,0.3]|       0.0|
|        5.0|       3.4|        1.5|       0.2|Iris-setosa|[5.0,3.4,1.5,0.2]|       0.0|
|        4.4|       2

In [0]:
irisTreino, irisTeste = irisas.randomSplit([0.7,0.3])
print(irisTreino.count())
print(irisTeste.count())

108
42


In [0]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [0]:
mlp = MultilayerPerceptronClassifier(maxIter=100, layers=[4, 5, 4, 3], featuresCol="independente", labelCol="dependente")
modelo = mlp.fit(irisTreino)

In [0]:
previsao = modelo.transform(irisTeste)
previsao.select("dependente","prediction").show(36)

+----------+----------+
|dependente|prediction|
+----------+----------+
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       2.0|       2.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       2.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       1.0|       2.0|
|       1.0|       2.0|
|       0.0|       0.0|
|       1.0|       2.0|
|       2.0|       2.0|
|       1.0|       2.0|
|       1.0|       2.0|
|       0.0|       0.0|
|       1.0|       2.0|
|       1.0|       2.0|
|       2.0|       2.0|
|       1.0|       2.0|
|       1.0|       2.0|
|       1.0|       2.0|
|       1.0|       2.0|
|       2.0|       2.0|
|       2.0|       2.0|
|       2.0|       2.0|
|       1.0|       2.0|
|       1.0|       2.0|
|       2.0|       2.0|
|       2.0|       2.0|
|       2.0|       2.0|
+----------+----------+
only showing top 36 rows



In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
performance = MulticlassClassificationEvaluator(labelCol="dependente", predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(previsao)
print(acuracia)

0.6428571428571429
